# Load libraries

In [1]:
from typing import List
import pandas as pd
import os
import numpy as np
from itertools import product
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from src.utils.memory_managment import downcast
le = LabelEncoder()
import category_encoders as ce
import warnings
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 40)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\feder\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\feder\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load Data

In [2]:
DATA_FOLDER = '../../data/'
ENGLISH_DATA_FOLDER = '../../data_english/'
CUSTOM_DATA_FOLDER = '../../data_custom/'
SUBMISSION_FOLDER = "../../submissions/"

In [3]:
transactions    = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'sales_train_v2.csv'))

In [4]:
items           = pd.read_csv(os.path.join(os.getcwd(), ENGLISH_DATA_FOLDER, 'items.csv'))
categories      = pd.read_csv(os.path.join(os.getcwd(), ENGLISH_DATA_FOLDER, 'categories.csv'))
test            = pd.read_csv(os.path.join(os.getcwd(), DATA_FOLDER, 'test.csv'))

In [5]:
shops           = pd.read_csv(os.path.join(os.getcwd(), ENGLISH_DATA_FOLDER, 'shops.csv'))

## Process categories

In [6]:
categories.head(10)

,category_name,category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4
5,Accessories - PSVita,5
6,Accessories - XBOX 360,6
7,Accessories - XBOX ONE,7
8,Tickets (Digital),8
9,Delivery of goods,9


looking at the categories it is clear the pattern "main category - subcategory". It should be extracted.


In [7]:
categories["main_category"] = categories["category_name"].apply(lambda x: x.split("-")[0].strip())
categories["main_category_id"] = le.fit_transform(categories["main_category"])

## Process Items

In [8]:
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def process_text(s: str) -> str:
    tokens = word_tokenize(s)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    words = [w for w in words if not w in stop_words]
    return " ".join(words)

In [9]:
processed_text_series = items["item_name"].apply(process_text)

In [10]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(processed_text_series.values)]
vector_size: int = 10
model = Doc2Vec(tagged_data, vector_size = vector_size, window = 2, min_count = 1, epochs = 100)

In [11]:
columns_vector_name = ["item_name_vector_{}".format(i) for i in range(vector_size)]
items[columns_vector_name] = pd.DataFrame(processed_text_series.apply(lambda s: model.infer_vector(word_tokenize(s))).tolist())
del model
del tagged_data
del processed_text_series

In [12]:
items = items.merge(categories, on='category_id')
items.sample(10)

,item_id,category_id,item_name,item_name_vector_0,item_name_vector_1,item_name_vector_2,item_name_vector_3,item_name_vector_4,item_name_vector_5,item_name_vector_6,item_name_vector_7,item_name_vector_8,item_name_vector_9,category_name,main_category,main_category_id
12318,10594,43,Gippenreiter Yu.B. Introduction to g...,-0.049158,-0.018023,-0.018866,-0.012253,-0.040055,0.022418,0.024388,0.002456,-0.021710,-0.046354,Books - Audiobooks,Books,5
2364,14029,40,BEST HISTORICAL BLOCKBUSTERS (2DVD),0.022326,-0.023248,-0.046411,-0.008843,0.011552,0.033437,-0.019132,-0.045087,-0.023020,-0.043262,Cinema - DVD,Cinema,6
18431,7832,22,"WipEout 2048 [PS Vita, Russian version]",0.047710,-0.038287,0.042359,-0.048915,-0.024975,-0.046228,0.020477,-0.014994,0.046253,-0.037153,Games - PSVita,Games,9
17756,10216,28,"The Witcher. Gift Edition [PC, Jewel...",0.035785,0.034584,0.002797,0.027241,-0.017777,-0.041783,0.034200,0.045799,0.034319,0.007924,PC Games - Additional Editions,PC Games,14
11847,8853,57,BI-2 CD 2 (mp3-CD) (Jewel),-0.040241,0.043671,0.039836,-0.023263,0.035877,-0.037120,0.045179,0.013408,0.029222,0.014668,Music - MP3,Music,12
3439,17336,40,GHOST (ANGEL WATERFALL AS A GIFT),-0.005664,-0.041995,0.022874,-0.002266,0.048207,-0.008125,0.043937,-0.012842,-0.031235,-0.032194,Cinema - DVD,Cinema,6
19662,15268,63,Soft toy Dragons Large plush Bezubik,0.017129,0.048479,-0.007098,0.001414,-0.000431,-0.021547,0.036118,0.025165,0.025426,0.023444,Gifts - Soft toys,Gifts,10
11533,12668,41,COLLECTION OF DE NIRO (CASINO / SLEE...,0.044478,-0.000681,0.021093,0.029773,0.048869,0.019271,-0.010663,0.019929,0.043921,-0.012690,Cinema - Collector's,Cinema,6
20627,16163,65,"Board Game Munchkin Secret Mine, art...",0.037379,0.047699,-0.016147,-0.027883,-0.047452,0.021445,0.014962,-0.043472,-0.041222,-0.015591,Gifts - Board games (compact),Gifts,10
7990,3869,19,"Hitman Absolution [PS3, Russian vers...",0.041819,0.041644,-0.028456,-0.038603,-0.013641,0.020857,0.000905,0.028626,0.049988,0.048395,Games - PS3,Games,9


Find possible duplicates in item id

In [13]:
duplicated_items = items[items.duplicated(["item_name", "category_id"], keep=False)]
duplicated_items["in_test"] = duplicated_items["item_id"].isin(test["item_id"].unique())
duplicated_items = duplicated_items.groupby('item_name').agg({'item_id':['first','last'],'in_test':['first','last']})

#if both item id's are in the test set do nothing
duplicated_items = duplicated_items[(duplicated_items[('in_test', 'first')]==False) | (duplicated_items[('in_test', 'last')]==False)]
#if only the first id is in the test set assign this id to both
temp = duplicated_items[duplicated_items[('in_test', 'first')]==True]
keep_first = dict(zip(temp[('item_id', 'last')], temp[('item_id',  'first')]))
#if neither id or only the second id is in the test set, assign the second id to both
temp = duplicated_items[duplicated_items[('in_test', 'first')]==False]
keep_second = dict(zip(temp[('item_id', 'first')], temp[('item_id',  'last')]))
item_map = {**keep_first, **keep_second}
del temp
del keep_first
del keep_second
del duplicated_items

In [14]:
item_map

{9802: 9767,
 11689: 11688,
 16628: 13804,
 15708: 15698,
 10840: 10841,
 2037: 2038,
 14537: 14539,
 18933: 19135,
 2968: 2970,
 10791: 19240,
 11604: 16716,
 9921: 14050,
 14044: 14045,
 11794: 15793,
 16309: 16314,
 16495: 16509,
 17202: 17203,
 19465: 19475,
 19579: 19581,
 15700: 15709,
 15660: 15673}

## Transactions analysis


In [15]:
transactions = transactions\
    .query('0 < item_price < 50000 and 0 < item_cnt_day < 1001')\
    .replace({
        'shop_id':{0:57, 1:58, 11:10}, #replacing obsolete shop id's
        'item_id':item_map #fixing duplicate item id's  
    })    

In [16]:
#removing shops which don't appear in the test set
transactions = transactions[transactions['shop_id'].isin(test.shop_id.unique())]

transactions['date'] = pd.to_datetime(transactions.date,format='%d.%m.%Y')
transactions['year'] = transactions.date.dt.year

# get percentage of sales of the shop in the specific month of the year
sales_year = transactions.groupby(["shop_id", "year"]).agg({"item_cnt_day": "sum"}).rename(columns={'item_cnt_day':'item_cnt_year'})
sales_month = transactions.groupby(["shop_id", "date_block_num"]).agg({"item_cnt_day": "sum", "year": "first"}).rename(columns={'item_cnt_day':'item_cnt_month'})
sales = sales_month.join(sales_year, on=["shop_id", "year"])
sales["perc_sales_year_shop"] = sales["item_cnt_month"] / sales["item_cnt_year"]
sales.drop(["item_cnt_month", "item_cnt_year", "year"], axis=1, inplace=True)
del sales_year
del sales_month

# get percentage of sales of a specific item in that month, considering all shops
sales_year = transactions.groupby(["item_id", "year"]).agg({"item_cnt_day": "sum"}).rename(columns={'item_cnt_day':'item_cnt_year'})
sales_month = transactions.groupby(["item_id", "date_block_num"]).agg({"item_cnt_day": "sum", "year": "first"}).rename(columns={'item_cnt_day':'item_cnt_month'})
sales_item = sales_month.join(sales_year, on=["item_id", "year"])
sales_item["perc_sales_year_item"] = sales_item["item_cnt_month"] / sales_item["item_cnt_year"]
sales_item.drop(["item_cnt_month", "item_cnt_year", "year"], axis=1, inplace=True)
del sales_year
del sales_month


In [17]:
# add tot number of that specific items sold
total_item_sold = transactions.groupby("item_id").agg({"item_cnt_day": "sum"}).rename(columns={"item_cnt_day": "total_item_sold"})
transactions = transactions.merge(total_item_sold, on="item_id")
del total_item_sold

# add tot number of items sold in a shop
total_item_sold_in_shop = transactions.groupby("shop_id").agg({"item_cnt_day": "sum"}).rename(columns={"item_cnt_day": "total_item_sold_in_shop"})
transactions = transactions.merge(total_item_sold_in_shop, on="shop_id")
del total_item_sold_in_shop

#revenue is needed to accurately calculate prices after grouping
transactions['revenue'] = transactions['item_cnt_day']*transactions['item_price']

Group by month and make a percentage of sells in the year

In [18]:
transactions = (transactions
     .groupby(['date_block_num', 'shop_id', 'item_id'])
     .agg({
         'item_cnt_day':'sum', 
         'revenue':'sum'
     })
     .reset_index()
     .rename(columns={'item_cnt_day':'item_cnt_month'})
     .merge(sales, on=["shop_id", "date_block_num"])
     .merge(sales_item, on=["item_id", "date_block_num"])
)

del sales
del sales_item
transactions.sample(5)

,date_block_num,shop_id,item_id,item_cnt_month,revenue,perc_sales_year_shop,perc_sales_year_item
731796,16,59,12321,1.0,149.0,0.073112,0.061453
504187,11,22,15851,1.0,699.0,0.097279,0.742424
1224275,30,6,16937,1.0,499.0,0.080144,0.161290
1228742,30,47,11188,2.0,898.0,0.087380,0.117647
101342,2,45,7895,3.0,2397.0,0.096021,0.073529


## Creation of all_data dataframe

In [19]:
all_data = [] 
for block_num in transactions['date_block_num'].unique():
    cur_shops = transactions.loc[transactions['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = transactions.loc[transactions['date_block_num'] == block_num, 'item_id'].unique()
    all_data.append(np.array(list(product(*[cur_shops, cur_items, [block_num]]))))

all_data = pd.DataFrame(np.vstack(all_data), columns=['shop_id', 'item_id', 'date_block_num'])
all_data.head()

,shop_id,item_id,date_block_num
0,2,27,0
1,2,33,0
2,2,317,0
3,2,438,0
4,2,471,0


In [20]:
#add the appropriate date_block_num value to the test set
test['date_block_num'] = 34
del test['ID']

In [21]:
#append test set to training dataframe
all_data = pd.concat([all_data,test]).fillna(0)
all_data = all_data.reset_index()
del all_data['index']

In [22]:
#join items information to the training dataframe
all_data = pd.merge(all_data, transactions, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)
all_data = pd.merge(all_data, items.drop(columns=['item_name','category_name']), on='item_id', how='left')

del transactions
del items

## Shop Analysis


In [23]:
#clustering shops by categores of item sold
shops_cats = pd.DataFrame(
    np.array(list(product(*[all_data['shop_id'].unique(), all_data['category_id'].unique()]))),
    columns =['shop_id', 'category_id']
)
temp = all_data.groupby(['category_id', 'shop_id']).agg({'item_cnt_month':'sum'}).reset_index()
temp2 = temp.groupby('shop_id').agg({'item_cnt_month':'sum'}).rename(columns={'item_cnt_month':'shop_total'})
temp = temp.join(temp2, on='shop_id')
temp['category_proportion'] = temp['item_cnt_month']/temp['shop_total']
temp = temp[['shop_id', 'category_id', 'category_proportion']]
del temp2
shops_cats = shops_cats.merge(temp, on=['shop_id','category_id'], how='left')
shops_cats = shops_cats.fillna(0)
del temp

shops_cats = shops_cats.pivot(index='shop_id', columns=['category_id'])
kmeans = KMeans(n_clusters=7, random_state=0).fit(shops_cats)
shops_cats['shop_cluster'] = kmeans.labels_.astype('int8')
del kmeans

#adding these clusters to the shops dataframe
shops = shops.join(shops_cats['shop_cluster'], on='shop_id')
del shops_cats

# remove shops that are not present in train or test
shops.dropna(inplace=True)


In [24]:
# insert shop name
shops["shop_city"] = shops["shop_name"].apply(lambda x: process_text(x).split(" ")[0])

#creating a column for the type of shop
shops["shop_type"] = "regular"

#there is some overlap in tc and mall, mall is given precedence
shops.loc[shops["shop_name"].str.contains(r"tc"), "shop_type"] = "tc"
shops.loc[shops["shop_name"].str.contains(r"mall|center|mega"), "shop_type"] = "mall"
shops.loc[shops["shop_id"].isin([9,20]), "shop_type"] = "special"
shops.loc[shops["shop_id"].isin([12,55]), "shop_type"] = "online"

shops["shop_city"] = le.fit_transform(shops["shop_city"].values)
shops["shop_type"] = le.fit_transform(shops["shop_type"].values)

shops["shop_cluster"] = shops["shop_cluster"].astype("int")

In [25]:
shops.sample(10)


,shop_name,shop_id,shop_cluster,shop_city,shop_type
45,Samara ParkHouse shopping center,45,0,16,0
49,"Tyumen SEC ""Crystal""",49,3,21,2
56,"Chekhov TC"" Karnaval """,56,2,2,2
6,"Voronezh (Plekhanovskaya, 13)",6,0,25,2
19,"Kursk TC "" Pushkinskiy """,19,0,9,2
37,"Novosibirsk SEC"" Mega """,37,0,12,2
31,"Moscow TC"" Semenovsky """,31,2,10,2
36,"Novosibirsk SEC"" Gallery Novosibirsk """,36,3,12,2
52,"Ufa TC"" Central """,52,0,22,2
26,"Moscow TTS"" Areal ""(Belyaevo)",26,0,10,2


In [26]:
#add shop information to the training dataframe
all_data = pd.merge(all_data, shops.drop(columns='shop_name'), on='shop_id', how='left')
del shops
all_data.head()

,shop_id,item_id,date_block_num,item_cnt_month,revenue,perc_sales_year_shop,perc_sales_year_item,category_id,item_name_vector_0,item_name_vector_1,item_name_vector_2,item_name_vector_3,item_name_vector_4,item_name_vector_5,item_name_vector_6,item_name_vector_7,item_name_vector_8,item_name_vector_9,main_category,main_category_id,shop_cluster,shop_city,shop_type
0,2,27,0,1.0,2499.0,0.115227,0.218750,19,0.009564,-0.022983,-0.036763,-0.042400,0.009389,0.030400,-0.018024,0.045419,-0.025249,-0.007084,Games,9,4,0,2
1,2,33,0,1.0,499.0,0.115227,0.127479,37,0.004434,-0.029593,-0.026342,-0.028270,-0.037854,0.048597,-0.000605,0.007260,-0.020710,0.027511,Cinema,6,4,0,2
2,2,317,0,1.0,299.0,0.115227,0.181818,45,-0.198342,0.466214,-0.194764,0.209400,0.565167,-0.083701,0.161291,-0.183867,-0.272822,-0.355384,Books,5,4,0,2
3,2,438,0,1.0,299.0,0.115227,0.150000,45,-0.024946,-0.025346,-0.017227,-0.003779,-0.024940,0.004191,-0.039311,0.043764,-0.040039,0.031280,Books,5,4,0,2
4,2,471,0,2.0,798.0,0.115227,0.163522,49,0.002027,0.004761,0.036307,0.007189,-0.030771,0.043258,0.014054,0.005688,-0.004921,0.033122,Books,5,4,0,2


In [27]:
# clip values of predicted month sales
all_data["item_cnt_month_unclipped"] = all_data["item_cnt_month"]
all_data["item_cnt_month"] = all_data["item_cnt_month"].clip(0, 20)

In [28]:
# downcast the dataframe
all_data = downcast(all_data)

In [29]:
#These features show how many months have passed since the first appearance of the item/name/category/group/shop
all_data['item_age'] = (all_data['date_block_num'] - all_data.groupby('item_id')['date_block_num'].transform('min')).astype('int8')
all_data['category_age'] = (all_data['date_block_num'] - all_data.groupby('category_id')['date_block_num'].transform('min')).astype('int8')
all_data['main_category_age'] = (all_data['date_block_num'] - all_data.groupby('main_category_id')['date_block_num'].transform('min')).astype('int8')
all_data['shop_age'] = (all_data['date_block_num'] - all_data.groupby('shop_id')['date_block_num'].transform('min')).astype('int8')


In [30]:
#indicates whether shops have previously sold the item
temp = all_data.query('item_cnt_month > 0').groupby(['item_id','shop_id']).agg({'date_block_num':'min'}).reset_index()
temp.columns = ['item_id', 'shop_id', 'item_shop_first_sale']
all_data = pd.merge(all_data, temp, on=['item_id','shop_id'], how='left')
del temp
all_data['item_shop_first_sale'] = all_data['item_shop_first_sale'].fillna(50)
#item age that stays at 0 if a shop hasn't sold the item
all_data['item_age_if_shop_sale'] = (all_data['date_block_num'] > all_data['item_shop_first_sale']) * all_data['item_age']
#the length of time an item has been for sale without being sold at individual shops
all_data['item_age_without_shop_sale'] = (all_data['date_block_num'] <= all_data['item_shop_first_sale']) * all_data['item_age']
del all_data['item_shop_first_sale']

In [31]:
#The target variable, 'item_cnt', is the monthly sale count of individual items at individual shops. We now create features showing average monthly sales based on various groupings

def agg_cnt_col(df, merging_cols, new_col,aggregation):
    temp = df.groupby(merging_cols).agg(aggregation).reset_index()
    temp.columns = merging_cols + [new_col]
    df = pd.merge(df, temp, on=merging_cols, how='left')
    return df

In [32]:
#individual items across all shops
all_data = agg_cnt_col(all_data, ['date_block_num','item_id'],'item_cnt_month_all_shops',{'item_cnt_month':'mean'})
all_data = agg_cnt_col(all_data, ['date_block_num','item_id'],'item_cnt_month_all_shops_median',{'item_cnt_month':'median'}) 
#all items in category at individual shops
all_data = agg_cnt_col(all_data, ['date_block_num','category_id','shop_id'],'category_cnt',{'item_cnt_month':'mean'})
all_data = agg_cnt_col(all_data, ['date_block_num','category_id','shop_id'],'category_cnt_median',{'item_cnt_month':'median'}) 
#all items in category across all shops
all_data = agg_cnt_col(all_data, ['date_block_num','category_id'],'category_cnt_all_shops',{'item_cnt_month':'mean'})
all_data = agg_cnt_col(all_data, ['date_block_num','category_id'],'category_cnt_all_shops_median',{'item_cnt_month':'median'})
#all items in group
all_data = agg_cnt_col(all_data, ['date_block_num','main_category_id','shop_id'],'group_cnt',{'item_cnt_month':'mean'})
#all items in group across all shops
all_data = agg_cnt_col(all_data, ['date_block_num','main_category_id'],'group_cnt_all_shops',{'item_cnt_month':'mean'})
#all items at individual shops
all_data = agg_cnt_col(all_data, ['date_block_num','shop_id'],'shop_cnt',{'item_cnt_month':'mean'})
#all items at all shops within the city
all_data = agg_cnt_col(all_data, ['date_block_num','shop_city'],'city_cnt',{'item_cnt_month':'mean'})

In [33]:
def agg_price_col(df, merging_cols, new_col):
    temp = df.groupby(merging_cols).agg({'revenue':'sum','item_cnt_month_unclipped':'sum'}).reset_index()
    temp[new_col] = temp['revenue']/temp['item_cnt_month_unclipped']
    temp = temp[merging_cols + [new_col]]
    df = pd.merge(df, temp, on=merging_cols, how='left')
    return df

#average item price
all_data = agg_price_col(all_data,['date_block_num','item_id'],'item_price')
#average price of items in category
all_data = agg_price_col(all_data,['date_block_num','category_id'],'category_price')
#average price of all items
all_data = agg_price_col(all_data,['date_block_num'],'block_price')

In [34]:

# downcast the dataframe
all_data = downcast(all_data)

## Insert lagging features

In [35]:
def lag_feature(df, lag, col, merge_cols):
    """
    Add a column on the dataframe with a specific shift \n
    The name is the same as the base column with the suffix _lag{lag}
    Add automatically also the column with the differnece between the two values
    :param df: input dataframe
    :param lag: month shift
    :param col: column on which the shift is done
    :param merge_cols: 
    :return: the same input dataframe with new columns
    """
    temp = df[merge_cols + [col]]
    temp = temp.groupby(merge_cols).agg({f'{col}':'first'}).reset_index()
    temp.columns = merge_cols + [f'{col}_lag{lag}']
    temp['date_block_num'] += lag
    df = pd.merge(df, temp, on=merge_cols, how='left')
    df[f'{col}_lag{lag}'] = df[f'{col}_lag{lag}'].fillna(0).astype('float32')
    df[f'{col}_lag{lag}_diff'] = df[col] - df[f'{col}_lag{lag}']
    del temp
    return df

These features each have 3 lagged columns returned:

- lag1 shows the value of the prior month
- lag2 shows the value two months prior
- lag12 shows the value of 12 months earlier

Moreover all the features difference between the new and the base one are generated

In [36]:
lag_cols = [
    ['item_cnt_month', [1, 2, 12], ['date_block_num', 'shop_id', 'item_id']],
    ['item_cnt_month_unclipped', [1, 2, 12], ['date_block_num', 'shop_id', 'item_id']],
    ['item_cnt_month_all_shops', [1, 2, 12], ['date_block_num', 'item_id']],
    ['category_cnt', [1, 2, 12], ['date_block_num', 'shop_id', 'category_id']],
    ['category_cnt_all_shops', [1, 2, 12], ['date_block_num', 'shop_id', 'category_id']],
    ['group_cnt', [1, 2, 12], ['date_block_num', 'shop_id', 'main_category_id']],
    ['group_cnt_all_shops', [1, 2, 12], ['date_block_num', 'main_category_id']],
    ['shop_cnt', [1, 2, 12], ['date_block_num', 'shop_id']],
    ['city_cnt', [1, 2, 12], ['date_block_num', 'shop_city']],
]

lag_cols_dict = {}
for element, counters, agglomer in lag_cols:
    lag_cols_dict[element] = {"lag_counters": counters,
                              "agg_columns": agglomer}

In [37]:
for col in lag_cols_dict:
    lags = lag_cols_dict[col]["lag_counters"]
    merge_cols = lag_cols_dict[col]["agg_columns"]
    for lag in lags:
        all_data = lag_feature(all_data, lag, col, merge_cols)

In [38]:
# add ratio between the month and the previous year

def cols_ratio(row, col1, col2):
    if row[col2] == np.NaN:
        return np.Nan
    else:
        return row[col1] / row[col2]

#all_data['item_cnt_month_diff_perc'] = all_data.apply(lambda x: cols_ratio(x, 'item_cnt_month_unclipped', 'item_cnt_month_lag12'), axis=1)
#all_data['item_cnt_month_all_shops_diff_perc'] = all_data.apply(lambda x: cols_ratio(x, 'item_cnt_month_all_shops', 'item_cnt_month_all_shops_lag12'), axis=1)
#all_data['category_cnt_diff_perc'] = all_data.apply(lambda x: cols_ratio(x, 'category_cnt', 'category_cnt_lag12'), axis=1)
#all_data['category_cnt_all_shops_diff_perc'] = all_data.apply(lambda x: cols_ratio(x, 'category_cnt_all_shops', 'category_cnt_all_shops_lag12'), axis=1)


all_data['item_cnt_month_diff'] = all_data['item_cnt_month_unclipped_lag1']/all_data['item_cnt_month_lag12']
all_data['item_cnt_month_all_shops_diff'] = all_data['item_cnt_month_all_shops_lag1']/all_data['item_cnt_month_all_shops_lag12']
all_data['category_cnt_diff'] = all_data['category_cnt_lag1']/all_data['category_cnt_lag12']
all_data['category_cnt_all_shops_diff'] = all_data['category_cnt_all_shops_lag1']/all_data['category_cnt_all_shops_lag12']

In [39]:
all_data = downcast(all_data)

In [40]:
all_data.drop("main_category",axis=1, inplace=True)

In [41]:
all_data.dtypes

shop_id                                  int16
item_id                                  int16
date_block_num                           int16
item_cnt_month                         float32
revenue                                float32
perc_sales_year_shop                   float32
perc_sales_year_item                   float32
category_id                              int16
item_name_vector_0                     float32
item_name_vector_1                     float32
item_name_vector_2                     float32
item_name_vector_3                     float32
item_name_vector_4                     float32
item_name_vector_5                     float32
item_name_vector_6                     float32
item_name_vector_7                     float32
item_name_vector_8                     float32
item_name_vector_9                     float32
main_category_id                         int32
shop_cluster                             int32
shop_city                                int32
shop_type    

In [42]:
all_data.reset_index().to_feather(os.path.join(os.getcwd(), CUSTOM_DATA_FOLDER, 'all_data_preprocessed.feather'))

